<a href="https://colab.research.google.com/gist/lubin-rci/7599bfff56aa9d0d0d230e7923421540/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to the Yiedl Competition Quickstart Guide!

This notebook will walk you through the required steps to take part in Yiedl's Updown and Neutral competitions.

If you wish to run this without setting up a local Python and Jupyter environment, consider running this in Google Colab by clicking on the "Open in Colab" badge above.

Let's begin!

# Setup

## Install the `yiedl` package

Requires Python >= 3.11

In [ ]:
!pip install yiedl-ai

In [ ]:
import yiedl

# configure logging to ensure output is shown in Jupyter/Colab
import logging, sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, force=True)

## Set Demo Parameters

⚠ **Make sure to set `demo_mode` correctly before running the other cells below.**

- Demo mode: Set `demo_mode` to `True`.
- Production mode: Set `demo_mode` to `False` or skip this section **after restarting the kernel**. The `yiedl` package is set to production mode settings by default.


*Each challenge of the competition is only open on Mondays from around 0600 UTC.*

*Demo competitions are available for development purposes any time of the week.*

*Demo competitions run on the Polygon mainnet and use demo tokens.*



In [ ]:
demo_mode = True

if not demo_mode:
  yiedl.tools.UPDOWN_COMP.address = yiedl.settings.UPDOWN_ADDRESS
  yiedl.tools.NEUTRAL_COMP.address = yiedl.settings.NEUTRAL_ADDRESS
  yiedl.settings.TOKEN = yiedl.settings.TOKEN

else:
  demo_token_address = "0xEF9B76BB9D37db69540766e264C6A950D7583dc1"
  demo_neutral = "0x3d441c73859325e767088Bd295B024eaf655d5c8"
  demo_updown = "0xbAa3d046F8970674474EfDc0A503C3E624E916c8"
  yiedl.tools.UPDOWN_COMP.address = demo_updown
  yiedl.tools.NEUTRAL_COMP.address = demo_neutral
  yiedl.settings.TOKEN = demo_token_address

## User Account Keys

### Pinata JWT Token (for storing/pinning files on IPFS)

`jwt`: This is a JSON Web Token (JWT) string that you can generate for free by creating a Pinata account. This will allow you to upload and pin files to IPFS, which is required for keeping submissions decentralized.

[View this guide](https://docs.yiedl.ai/rci-competition/data-scientists/getting-started-via-automated-submissions/create-and-import-free-pinata-account) for a more visual set of instructions. Otherwise, follow the steps below:

   1. Head over to [https://pinata.cloud](https://pinata.cloud) and sign up for a free account.
   2. Log in to your [account's API key page](https://app.pinata.cloud/developers/api-keys).
   3. Click on `+ New Key` on the top right.
   4. Under `Key Name`, give the API Key a name, such as "yiedl-submissions".
   5. Under "Customize Permissions" > "Legacy Endpoints" > "Pinning", check the box for `pinFileToIPFS`.
   6. Leave all other fields as default.
   7. **Copy the entire string in the `JWT` field and store it somewhere safe.** (The string should look something like "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.ey.......")
   8. Load this string into the `jwt` variable in the below cell.

In [ ]:
jwt: str = ''

# # example: loading from Google Colab Secrets (left panel)
# from google.colab import userdata
# jwt = userdata.get('jwt')

### Wallet Account Keys

`address`: This is your wallet address. It should be a hex string beginning with "0x" followed by 40 characters.

`pk`: This is the private key to your wallet address.

*The private key can generally be found via the following steps. Some details may differ depending on the wallet version.*

[View this guide](https://docs.yiedl.ai/rci-competition/data-scientists/getting-started-via-automated-submissions/create-and-import-polygon-account) for a more visual set of instructions for creating and importing the address and private key.

Otherwise, follow the steps below. These assume that you already have a wallet set up with [Phantom](https://phantom.com/download) or [Metamask](https://metamask.io/).

  - "Phantom Wallet": You can find the private key from the following steps:
    1. Click on Phantom Wallet extension.
    2. Click on top left account icon in the wallet window;
    3. Click on settings icon at the bottom of the bar that appears;
    4. Click on "Manage accounts:;
    5. Select the desired account;
    6. Click on "Show Private Key";
    7. Enter your password;
    8. Click on "Polygon";
    9. Click on disclaimer checkboxes and "Continue";
    10. Copy password and store it somewhere safe.
    11. Load this into the `pk` variable.

  - "Metamask": You can find the private key from the following steps:
    1. Click on Metamask Wallet extension;
    2. Click on the account name on the top left;
    3. Click on the 3-dot menu next to your desired account;
    4. Click on "Account Details";
    5. Click on "Private Keys";
    6. Enter your passwoard;
    7. Copy your private key for "Polygon" and store it somewhere safe.
    8. Load this into the `pk` variable.

⚠ **! Please keep your Private Key safe !**

Stolen private keys lead to loss of funds from your blockchain wallet.

Please take necessary steps to make sure your private key string is never exposed.

This can include storing the key as a password-protected local environment variable, as well as keeping your computer safe from prying eyes or malware.

If you are following this guide in a public area, please consider creating a separate account in a more private setting for use in your production code.

In [ ]:
address = ""

pk = ""


# # example: loading from Google Colab Secrets (left panel)
# from google.colab import userdata
# pk = userdata.get('private_key')


In [ ]:
# Check that the provided address belongs to the private key
derived_address = yiedl.tools.get_account_address(pk)
assert derived_address.lower() == address.lower(), "The provided address does not match the private key."

## Initialise submitters
The same wallet is used to initialise both submitters but we initialise 2 submitter instances: one for each competition.

In [ ]:
updown_submitter = yiedl.Submitter(jwt, address, yiedl.tools.CompetitionIds.UPDOWN, pk)
neutral_submitter = yiedl.Submitter(jwt, address, yiedl.tools.CompetitionIds.NEUTRAL, pk)

# Download Weekly Dataset

The weekly dataset is the same for both competitions, so we can use either submitter instance to perform the download.

By default, the datasets are downloaded and unzipped to the "datasets/weekly_dataset" folder.

In [ ]:
dataset_dir = updown_submitter.download_and_unzip_weekly_dataset()

## Download Daily Dataset (optional)

A daily dataset is also available.
The daily dataset is the same for both competitions, so we can use either submitter instance to perform the download.
By default, the datasets are downloaded and unzipped to the "datasets/daily_dataset" folder.

The daily dataset is much larger than the weekly one - for the purposes of this quickstart guide, we will stick to the weekly dataset.

In [ ]:
# daily_dataset_dir = updown_submitter.download_and_unzip_daily_dataset()

# Generate predictions from dataset

The following is an example to demonstrate what needs to go into the csv file used for submission to both the *Neutral* and *Updown* competitions.

## Data Setup

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import os
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

In [ ]:

train_dataset_path = os.path.join(dataset_dir, yiedl.settings.YIEDL_TRAIN_FILE_PATH)
validation_dataset_path = os.path.join(dataset_dir, yiedl.settings.YIEDL_VALIDATION_FILE_PATH)

train_dataset  = pd.read_csv(train_dataset_path, index_col = 'date')
validation_dataset  = pd.read_csv(validation_dataset_path, index_col = 'date')

In [ ]:
# first column (symbol) is the ticker
# 'target_updown' is the return of a given crypto, such that final_price / initial_price - 1
# 'target_neutral' is the rank by Era using 'target_updown'
# the rest of the columns are features to be used for ML
train_dataset.head()

In [ ]:
# validation_dataset is the latest data from most recent Era
# it has the same structure as the dataset, however 'target_updown' and 'target_neutral' are NaN
validation_dataset.head()

### Create X and y from dataset
- Currently there are 2 competitions user could participate:
- (i) Updown (using target_updown as target)
- (ii) Market Neutral (using target_neutral as target)

In [ ]:
# X is all the columns except the 'symbol', 'target_updown', 'target_neutral'
X = train_dataset.iloc[:, 1:-2]

# y is just the target
y_updown = train_dataset.target_updown
y_neutral = train_dataset.target_neutral

# symbols
symbols = train_dataset.symbol

In [ ]:
X.head()

In [ ]:
y_updown.head()

In [ ]:
y_neutral.head()

### Split X, y into train and test

In [ ]:
train = 0.9
test = 0.1
era = len(X.index.unique())
train_era = int(era * train)
test_era = era - train_era
print('total Era: {}'.format(era))
print('train Era: {}'.format(train_era))
print('test Era: {}'.format(test_era))


In [ ]:
#split train and test set according to the train_era and test_era
X_train = X[X.index < X.index.unique()[train_era]]
y_updown_train = y_updown[y_updown.index < y_updown.index.unique()[train_era]]
y_neutral_train = y_neutral[y_neutral.index < y_neutral.index.unique()[train_era]]
symbols_train = symbols[symbols.index < symbols.index.unique()[train_era]]

X_test = X[X.index >= X.index.unique()[train_era]]
y_updown_test = y_updown[y_updown.index >= y_updown.index.unique()[train_era]]
y_neutral_test = y_neutral[y_neutral.index >= y_neutral.index.unique()[train_era]]
symbols_test = symbols[symbols.index >= symbols.index.unique()[train_era]]

print('X_train shape: {}'.format(X_train.shape))
print('X_test shape: {}'.format(X_test.shape))


## Tradeable Symbol List
The following [77 symbols](https://docs.yiedl.ai/rci-competition/submission-guide#id-1.7-symbols-used-to-measure-the-goodness-of-submissions) are used for evaluation for both *Neutral* and *Updown* competitions.


In [ ]:
tradeable_symbol_list = ['AAVE', 'ADA', 'ALGO', 'APE', 'APT', 'ATOM', 'AVAX', 'AXS', 'BAL', 'BCH', 'BLUR', 'BNB', 'BONK', 'BTC', 'C98', 'CAKE', 'COMP', 'CRV', 'CVX', 'DAI', 'DAO', 'DOGE', 'DOT', 'DYDX', 'EOS', 'ETC', 'ETH', 'FET', 'FIL', 'FRAX', 'FTM', 'FXS', 'GRT', 'HAI', 'HERO', 'ICP', 'IMX', 'INJ', 'LDO', 'LINK', 'LTC', 'MATIC', 'MAV', 'MBOX', 'MKR', 'MOON', 'NEAR', 'OP', 'PENDLE', 'PEPE', 'PERP', 'PIT', 'POLS', 'PYTH', 'QUACK', 'RDNT', 'RNDR', 'RPL', 'RUNE', 'SFP', 'SFUND', 'SHIB', 'SOL', 'STG', 'SUI', 'SUSHI', 'TIA', 'TOKEN', 'TRX', 'UNI', 'USDT', 'XLM', 'XMR', 'XRP', 'XTZ', 'XVS', 'YFI']

## For NEUTRAL competition

### We use simple Linear Regression to train a model and check the Spearman correlation

In [ ]:
from sklearn.linear_model import LinearRegression

reg_market_neutral = LinearRegression(n_jobs=-1).fit(X_train, y_neutral_train)

In [ ]:
# function to calculate Spearman correlation by era (mean, std, max, min)

def spearman_by_era(prediction, target):
    df = pd.DataFrame(index=target.index,
                    data = {'prediction': prediction,
                            'target': target}
                    )

    spearman_era_list = []
    for era in df.index.unique():
        era_df = df[df.index == era]
        spearman_corr = sp.stats.spearmanr(era_df.prediction, era_df.target)[0]
        spearman_era_list.append(spearman_corr)

    mean = round(np.mean(spearman_era_list), 4)
    std = round(np.std(spearman_era_list), 4)
    max_val = round(np.max(spearman_era_list), 4)
    min_val = round(np.min(spearman_era_list), 4)
    return mean, std, max_val, min_val

# function to calculate portfolio return by era (mean, std, max, min)

def calculate_return(symbols, prediction, actual_return):
    df = pd.DataFrame(index=actual_return.index,
                    data = {'symbol': symbols,
                            'prediction': prediction,
                            'actual_return': actual_return}
                    )

    dfs = df[df.symbol.isin(tradeable_symbol_list)]

    return_era_list = []
    for era in dfs.index.unique():
        era_df = dfs[dfs.index == era]

        # re-rank predictions to build a market neutral strategy
        ranks = sp.stats.rankdata(era_df.prediction)
        norm_ranks = [(r - 1) / (len(ranks) - 1) for r in ranks]

        # compute allocations
        tot = sum((abs(2 * n - 1) for n in norm_ranks))
        allocations = [(2 * n - 1) / tot for n in norm_ranks]

        # compute gain as dot product between allocation and relative deltas
        pct_gain = np.dot(allocations, era_df.actual_return)

        return_era_list.append(pct_gain)

    mean = round(np.mean(return_era_list), 4)
    std = round(np.std(return_era_list), 4)
    max = round(np.max(return_era_list), 4)
    min = round(np.min(return_era_list), 4)
    return mean, std, max, min

y_pred_train = reg_market_neutral.predict(X_train)
train_spearman = spearman_by_era(y_pred_train, y_neutral_train)
print('Train dataset Spearman correlation by era: mean = {} ; std = {} ; max = {} ; min = {}'.format(train_spearman[0],
                                                                                             train_spearman[1],
                                                                                             train_spearman[2],
                                                                                             train_spearman[3]))

y_pred_test = reg_market_neutral.predict(X_test)
test_spearman = spearman_by_era(y_pred_test, y_neutral_test)
print('Test dataset Spearman correlation by era: mean = {} ; std = {} ; max = {} ; min = {}'.format(test_spearman[0],
                                                                                             test_spearman[1],
                                                                                             test_spearman[2],
                                                                                             test_spearman[3]))

train_return = calculate_return(symbols_train, y_pred_train, y_updown_train)
print('Train dataset return by era: mean = {} ; std = {} ; max = {} ; min = {}'.format(train_return[0],
                                                                                 train_return[1],
                                                                                 train_return[2],
                                                                                 train_return[3]))

test_return = calculate_return(symbols_test, y_pred_test, y_updown_test)
print('Test dataset return by era: mean = {} ; std = {} ; max = {} ; min = {}'.format(test_return[0],
                                                                                 test_return[1],
                                                                                 test_return[2],
                                                                                 test_return[3]))


In [ ]:
# Use the trained Linear Regression model to make prediction on latest data

X_validation = validation_dataset.iloc[:, 1:-2]
y_validation_market_neutral = reg_market_neutral.predict(X_validation)

y_validation_market_neutral[:5]

### Let's use the prediction from linear regression to join with validation dataset symbol for submission

In [ ]:
prediction_market_neutral = pd.DataFrame()
prediction_market_neutral['symbol'] = list(validation_dataset.symbol)
prediction_market_neutral['prediction'] = y_validation_market_neutral
prediction_market_neutral = prediction_market_neutral[prediction_market_neutral.symbol.isin(tradeable_symbol_list)]

### Check is the prediction in accordance for submission format

In [ ]:
#check if y_latest is in accordance to shape for submission and required symbols
if set(prediction_market_neutral.symbol) == set(tradeable_symbol_list):
    print('symbol matched!')
else:
    print('symbol unmatched, the symbol in prediction df must match the symbol in validation_dataset...')

if prediction_market_neutral.shape[1] == 2:
    print('column counts ok!')
else:
    print('It should have 2 columns, first column with symbol, second with prediction...')


### Output prediction as a .csv file for submission

In [ ]:
neutral_submission_file_path = neutral_submitter.save_df_to_csv(prediction_market_neutral)

## For UPDOWN competition

### For demonstration we could also use Linear Regression to train a model and check the RMSE

In [ ]:
reg_updown = LinearRegression(n_jobs=-1).fit(X_train, y_updown_train)

In [ ]:
# function to calculate RMSE by era (mean, std, max, min)

from sklearn.metrics import mean_squared_error

def rmse_by_era(prediction, target):
    df = pd.DataFrame(index=target.index,
                    data = {'prediction': prediction,
                            'target': target}
                    )

    rmse_era_list = []
    for era in df.index.unique():
        era_df = df[df.index == era]
        mse = mean_squared_error(era_df.target, era_df.prediction)
        rmse = np.sqrt(mse)
        rmse_era_list.append(rmse)

    mean = round(np.mean(rmse_era_list), 4)
    std = round(np.std(rmse_era_list), 4)
    max_val = round(np.max(rmse_era_list), 4)
    min_val = round(np.min(rmse_era_list), 4)
    return mean, std, max_val, min_val


y_pred_train = reg_updown.predict(X_train)
train_rmse_stat = rmse_by_era(y_pred_train, y_updown_train)
print('Train dataset RMSE by era: mean = {} ; std = {} ; max = {} ; min = {}'.format(train_rmse_stat[0],
                                                                             train_rmse_stat[1],
                                                                             train_rmse_stat[2],
                                                                             train_rmse_stat[3]))
y_pred_test = reg_updown.predict(X_test)
test_rmse_stat = rmse_by_era(y_pred_test, y_updown_test)
print('Test dataset RMSE by era: mean = {} ; std = {} ; max = {} ; min = {}'.format(test_rmse_stat[0],
                                                                             test_rmse_stat[1],
                                                                             test_rmse_stat[2],
                                                                             test_rmse_stat[3]))

train_return = calculate_return(symbols_train, y_pred_train, y_updown_train)
print('Train dataset return by era: mean = {} ; std = {} ; max = {} ; min = {}'.format(train_return[0],
                                                                                 train_return[1],
                                                                                 train_return[2],
                                                                                 train_return[3]))

test_return = calculate_return(symbols_test, y_pred_test, y_updown_test)
print('Test dataset return by era: mean = {} ; std = {} ; max = {} ; min = {}'.format(test_return[0],
                                                                                 test_return[1],
                                                                                 test_return[2],
                                                                                 test_return[3]))

In [ ]:
# Use the trained Linear Regression model to make prediction on latest data

X_validation = validation_dataset.iloc[:, 1:-2]
y_validation_updown = reg_updown.predict(X_validation)

y_validation_updown[:5]

### Let's use the prediction from linear regression to join with validation dataset symbol for submission

In [ ]:
prediction_updown = pd.DataFrame()
prediction_updown['symbol'] = list(validation_dataset.symbol)
prediction_updown['prediction'] = y_validation_updown
prediction_updown = prediction_updown[prediction_updown.symbol.isin(tradeable_symbol_list)]

### Check is the prediction in accordance for submission format

In [ ]:
#check if y_latest is in accordance to shape for submission
if set(prediction_updown.symbol) == set(tradeable_symbol_list):
    print('symbol matched!')
else:
    print('symbol unmatched, the symbol in prediction df must match the symbol in validation_dataset...')

if prediction_updown.shape[1] == 2:
    print('column counts ok!')
else:
    print('It should have 2 columns, first column with symbol, second with prediction...')


### Output prediction as a .csv file for submission!

In [ ]:
updown_submission_file_path = updown_submitter.save_df_to_csv(prediction_updown)

# Submit predictions to the 2 competitions on the Polygon blockchain

Staking and submitting predictions involve sending transactions to the blockchain and typically take around 1-2 minutes to complete.

## View POL balance for wallet

The same wallet is connected to both submitters in this notebook, so we can check the balances from either one.

In [ ]:
print(f"POL balance for {updown_submitter.address}: {updown_submitter.get_pol_balance()} POL")

## View YIEDL balance for wallet


In [ ]:
print(f"YIEDL balance for {updown_submitter.address}: {updown_submitter.get_yiedl_balance()} YIEDL")

## View YIEDL staked on UPDOWN Competition by wallet


In [ ]:
print(f"YIEDL staked on UPDOWN Competition by {updown_submitter.address}: {updown_submitter.get_stake()} YIEDL")

## View YIEDL staked on NEUTRAL Competition by wallet


In [ ]:
print(f"YIEDL staked on NEUTRAL Competition by {neutral_submitter.address}: {neutral_submitter.get_stake()} YIEDL")

## Stake and submit to `UPDOWN` Competition.

`stake_amount`: The final amount of YIEDL you want to be staked from your account.

Example:

- If your current stake is 100 YIEDL and you want to increase it to 120 YIEDL, set `stake_amount` to **120**. This will deposit 20 YIEDL from your account to the competition to be staked.

- If you current stake is 100 YIEDL and you want to withdraw 70 YIEDL, set `stake_amount` to **30**.


In [ ]:
stake_amount = 100.00
transaction_success = updown_submitter.stake_and_submit(stake_amount, updown_submission_file_path)
assert transaction_success, 'Stake and submit on UPDOWN failed.'
print("Stake and submit on UPDOWN completed.")

In [ ]:
print(f"YIEDL staked on UPDOWN Competition by {updown_submitter.address}: {updown_submitter.get_stake()} YIEDL")

## Stake and submit to `NEUTRAL` Competition.

In [ ]:
stake_amount = 100.00
transaction_success = neutral_submitter.stake_and_submit(stake_amount, neutral_submission_file_path)
assert transaction_success, 'Stake and submit on NEUTRAL failed.'
print("Stake and submit on NEUTRAL completed.")

In [ ]:
print(f"YIEDL staked on NEUTRAL Competition by {neutral_submitter.address}: {neutral_submitter.get_stake()} YIEDL")

The methods `.set_stake(<stake_amount>)` and `.submit(<submission_file_path>)` are also available if you wish to perform these actions seperately.

## Retrieve and double-check UPDOWN submission. (optional)
This section retrieves your submitted files, decrypts them, and compares them to the original file.

If the verification fails, please wait a few minutes and perform the verification again. If the problem persists, please re-submit your predictions.

In [ ]:
verification_success = updown_submitter.download_and_check(updown_submission_file_path)
assert verification_success, 'UPDOWN submission verification failed.'
print('Files are identical. UPDOWN verification check passed.')
updown_stake = updown_submitter.get_stake()
print(f'UPDOWN Stake: {updown_stake:.3f} YIEDL')

### Retrieve and double-check NEUTRAL submission. (optional)
This section retrieves your submitted files, decrypts them, and compares them to the original file.

If the verification fails, please wait a few minutes and perform the verification again. If the problem persists, please re-submit your predictions.

In [ ]:
verification_success = neutral_submitter.download_and_check(neutral_submission_file_path)
assert verification_success, 'NEUTRAL submission verification failed.'
print('Files are identical. NEUTRAL verification check passed.')
neutral_stake = neutral_submitter.get_stake()
print(f'NEUTRAL Stake: {neutral_stake:.3f} YIEDL')

### Withdraw Submissions (an example)

The following is an example of how to withdraw your stake and submission.


In [ ]:
withdraw_success = updown_submitter.withdraw()
assert withdraw_success, "Withdraw failed."
print("Withdraw from UPDOWN completed.")

updown_stake = updown_submitter.get_stake()
print(f'UPDOWN Stake: {updown_stake:.3f} YIEDL')

print(f"YIEDL balance for {updown_submitter.address} after withdrawal: {updown_submitter.get_yiedl_balance()} YIEDL")